In [2]:
import pandas as pd
import numpy as np
import json
from pyvis.network import Network


def flatten(l):
    return [item for sublist in l for item in sublist]

with open('data/ingredients.json') as f:
    ingredients_json = json.load(f)

with open('data/effects.json') as f:
    effects_json = json.load(f)

In [3]:
ingredients = [x['name'] for x in ingredients_json]
ingredients_dict = {x['name'] : x for x in ingredients_json}

In [4]:
all_effects_arr = np.array(list(effects_json.keys()))

ingredients_effects = {}

for ingredient in ingredients_json:
    effects_arr = np.array(ingredient['effects'])
    ingredients_effects[ingredient['name']] = list(np.isin(all_effects_arr, effects_arr))

db = pd.DataFrame(ingredients_effects)
db.index = all_effects_arr

In [5]:

attribute_effects_color = {
    'defense': '#2845d7', 
    'health': '#74e876', 
    'magicka': '#f882f1', 
    'other': '#a7abb0', 
    'skill': '#56C6FE', 
    'stamina': '#fce269'
}
type_effects_color = {
    'help' : '#23772D',
    'hurt' : '#950B22'
}

In [6]:
def create_network(network_ingredients, network_db, html_name, size_effect_by_value = False, size_ingredient_by_value = False, basic_ingredients = []):
  font_color = "white" if size_effect_by_value else "white"
  background_color = 'black' if size_effect_by_value else '#bee9dc7a'
  
  g = Network(height='850px', 
              font_color=font_color, 
              bgcolor=background_color,
              neighborhood_highlight=False,
              select_menu=True,
              filter_menu=False,
              layout=None
            )
  
  
  ingredient_node_shape = 'dot' if size_ingredient_by_value else 'ellipse'
  
  for x in network_ingredients:
    g.add_node('Ingredient: ' + x,
              label = '  ' + x + '  ',
              color = {
                  'border': '#f7ff92',
                  'background': '#855600',
                  'highlight': {
                    'border': '#fff800',
                    'background': '#454B53',
                  }},
              shape = ingredient_node_shape,
              size= 10 + ingredients_dict[x]['potionValue']/2 if size_ingredient_by_value else 10,
              basic = 1 if (basic_ingredients == [] or x in basic_ingredients) else 0,
                category = 'ingredient'
              )
  
  effect_node_shape = 'square' if size_effect_by_value else 'box'
  effect_size = lambda e: 10 + effects_json[e]['value']/2 if size_effect_by_value else 10
  for e in network_db.index:
    g.add_node('Effect: '+e,
                label = '\n    '+e+'    \n\n',
                color = {
                  'border': attribute_effects_color[effects_json[e]['attribute']],
                  'background': type_effects_color[effects_json[e]['type']],
                  'highlight': {
                    'border': type_effects_color[effects_json[e]['type']],
                    'background': attribute_effects_color[effects_json[e]['attribute']],
                  }},
                shape = effect_node_shape,
                borderWidthSelected = 6,
                borderWidth = 3,
                size = effect_size(e),
                category = 'effect'
                )
    
  edge_list = network_db.stack().loc[lambda x: x].index.tolist()
  g.add_edges([('Effect: '+edge[0],'Ingredient: '+edge[1],4,) for edge in edge_list])

  g.force_atlas_2based(
      gravity= -300,
      central_gravity = 0.01,
      spring_length= 200,
      spring_strength= 1,
      damping= 1,
      overlap= 0.9
  )
  g.show_buttons(filter_=["physics"])
  g.show(html_name, notebook=False)

In [7]:
current_ingredients=[
    "Blue Mountain Flower",
    "Bone Meal",
    "Charred Skeever Hide",
    "Chicken's Egg",
    "Deathbell",
    "Dragon's Tongue",
    "Elves Ear",
    "Fly Amanita",
    "Frost Mirriam",
    "Garlic",
    "Glowing Mushroom",
    "Lavender",
    "Mora Tapinella",
    "Mudcrab Chitin",
    "Nightshade",
    "Purple Mountain Flower",
    "Red Mountain Flower",
    "River Betty",
    "Rock Warbler Egg",
    "Salt Pile",
    "Silverside Perch",
    "Skeever Tail",
    "Slaughterfish Egg",
    "Slaughterfish Scales",
    "Snowberries",
    "Thistle Branch",
    "Tundra Cotton",
    "Wheat"
 ]
useful_db = db[current_ingredients][db[current_ingredients].sum(axis=1) >= 2]

In [8]:
create_network(current_ingredients, useful_db, 'HTMLGraphs/python_generated/my_ingredients.html')
create_network(ingredients, db, 'HTMLGraphs/python_generated/all_ingredients.html')

create_network(current_ingredients, useful_db, 'HTMLGraphs/python_generated/my_ingredients_by_value.html', True, True)
create_network(ingredients, db, 'HTMLGraphs/python_generated/all_ingredients_by_value.html', True, True)

HTMLGraphs/python_generated/my_ingredients.html
HTMLGraphs/python_generated/all_ingredients.html
HTMLGraphs/python_generated/my_ingredients_by_value.html
HTMLGraphs/python_generated/all_ingredients_by_value.html


In [9]:
# ing_by_effect = db.apply(lambda row: row.index[row].tolist(), axis=1)
# effect_by_ing = {i['name']: i['effects'] for i in ingredients_json}
